# Make inspection plots from HDF5 dataset
---
* Amplitude versus subband/frequency for both XX and YY with all stations on the same axes.
* Clock versus time for XX with all stations on the same axes.
* Clock difference between XX and YY versus time with all stations on the same axes.
* TEC versus time for XX with all stations on the same axes.
* TEC difference between XX and YY versus time with all stations on same axes.

A test dataset can be found [here](https://github.com/mooneyse/Jupyter-Notebooks.git).

In [139]:
''' amplitude versus frequency for XX and YY ============================================== '''
    
# import required modules ---------------------------------------------------------------------

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from losoto.h5parm import h5parm

# load the data -------------------------------------------------------------------------------

name = '/home/sean/test.h5'
f = h5parm(name, readonly = True)
amplitude = f.getSoltab('sol000', 'amplitude000')

# set up the plot -----------------------------------------------------------------------------

plt.figure(figsize = (24, 12))
normalised = [(frequency - min(amplitude.freq[:]))/(max(amplitude.freq[:]) - min(amplitude.freq[:])) for frequency in amplitude.freq[:]]

# plot the amplitude for each station ---------------------------------------------------------

# note: I think the plot is correct but all of the core stations have one massive peak that
#       is drowing out all of the other points on the plot so I added a step that clips all
#       values above a given threshold

threshold = 3

text = open('clippings.txt', 'w')
text.write('Clipping threshold: ' + str(threshold) + '\n')

# note: here I'm looping over all of the stations and for each polarisation I end up with a
#       list which has 843 elements - one for each time step - and each of those 843 time
#       steps is a list itself of 122 elements - one value for each subband

for a in range(len(amplitude.ant[:])):
    xx = []
    average_xx = []
    yy = []
    average_yy = []

# note: the xx list has the 843 elements and the average_xx list has 122 - one for each
#       subband which was found by averaging over the 843 time steps for each frequency
#       value, and this is done for the X and Y polarisations
    
    for value in amplitude.val[0, 0, a, :, :]: # [polarisation, direction, station, frequency, time]
        xx.append(value)

    for time in range(len(xx)):
        average_xx.append(np.average(xx[time]))
        
    for value in amplitude.val[1, 0, a, :, 0]:
        yy.append(value)

    for time in range(len(yy)):
        average_yy.append(np.average(yy[time]))

# note: this is where the clipping is actually done, where the values are saved to a new
#       list if they meet the criteria, and if they don't I add the most recent value that
#       did meet the criteria so the plot looks continuous (except where there is no recent
#       value and then 0 is added)
        
    count_x, count_y, clipped_xx, clipped_yy = [], [], [], []

    if len(average_xx) == len(average_yy):
        for clip in range(len(average_xx)):
            if average_xx[clip] < threshold:
                clipped_xx.append(np.average(average_xx[clip]))

            else:
                if len(clipped_xx) == 0:
                    clipped_xx.append(0)
                
                else:
                    clipped_xx.append(clipped_xx[clip - 1])

                count_x.append(0)

            if average_yy[clip] < threshold:
                clipped_yy.append(np.average(average_yy[clip]))

            else:
                if len(clipped_yy) == 0:
                    clipped_yy.append(0)
                
                else:
                    clipped_yy.append(clipped_yy[clip - 1])

                count_y.append(0)

    text.write('Station: ' + str(amplitude.ant[a])[2:-1] + '; Polarisation: ' + str(amplitude.pol[0])[2:-1] + '; Clipped: ' + str(len(count_x)) + '\n')
    text.write('Station: ' + str(amplitude.ant[a])[2:-1] + '; Polarisation: ' + str(amplitude.pol[1])[2:-1] + '; Clipped: ' + str(len(count_y)) + '\n')

    plt.plot(normalised, clipped_xx, lw = 0.5, label = str(amplitude.ant[a])[2:-1] + ' ' + str(amplitude.pol[0])[2:-1]) # one series per station per polarisation
    plt.plot(normalised, clipped_yy, lw = 0.5, label = str(amplitude.ant[a])[2:-1] + ' ' + str(amplitude.pol[1])[2:-1])

text.close()
    
# tidy up/ the plot ----------------------------------------------------------------------------

mpl.rcParams.update({'font.size': 10})
plt.xlim(min(normalised), max(normalised))
plt.title('Amplitude versus frequency for ' + str(amplitude.pol[0])[2:-1] + ' and ' + str(amplitude.pol[1])[2:-1] + ' polarisations', fontsize = 10)
plt.xlabel('Frequency (normalised)', fontsize = 10)
plt.ylabel(str(amplitude.pol[0])[2:-1] + ' and ' + str(amplitude.pol[1])[2:-1] + ' amplitude', fontsize = 10)
plt.legend(loc = 'upper center', bbox_to_anchor = (0.5, -0.05), ncol = 17, frameon = False, fontsize = 7)
plt.savefig('amplitude.png', bbox_inches = 'tight')
plt.close('all')

In [1]:
# have to average over frequency for these scripts

''' clock versus time for XX ============================================================== '''

# import required modules ---------------------------------------------------------------------

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from losoto.h5parm import h5parm

# load the data -------------------------------------------------------------------------------

name = '/home/sean/test.h5'
f = h5parm(name, readonly = True)
clock = f.getSoltab('sol000', 'clock000')

# set up the plot -----------------------------------------------------------------------------

p = 0 # 0 = XX; 1 = YY
plt.figure(figsize = (24, 12))
normalised = [(time - min(clock.time[:]))/(max(clock.time[:]) - min(clock.time[:])) for time in clock.time[:]]

# plot the clock values for each station ------------------------------------------------------

for a in (range(len(clock.ant[:]))):
    values = []

    for value in clock.val[:, a, p]:
        values.append(value * 1e9)

    plt.plot(normalised, values, lw = 0.5, label = str(clock.ant[a])[2:-1])

# tidy up the plot ----------------------------------------------------------------------------

mpl.rcParams.update({'font.size': 10})
plt.xlim(min(normalised), max(normalised))
plt.title('Clock versus time for the ' + str(clock.pol[p])[2:-1] + ' polarisation for all stations', fontsize = 10)
plt.xlabel('Time (normalised)', fontsize = 10)
plt.ylabel(r'Clock (ns)', fontsize = 10)
plt.legend(loc = 'upper center', bbox_to_anchor = (0.5, -0.05), ncol = 17, frameon = False, fontsize = 7)
plt.savefig('clock_xx.png', bbox_inches = 'tight')
plt.close('all')

In [2]:
''' clock difference between XX and YY versus time ======================================== '''

# import required modules ---------------------------------------------------------------------

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from losoto.h5parm import h5parm

# load the data -------------------------------------------------------------------------------

name = '/home/sean/test.h5'
f = h5parm(name, readonly = True)
clock = f.getSoltab('sol000', 'clock000')

# set up the plot -----------------------------------------------------------------------------

plt.figure(figsize = (24, 12))
normalised = [(time - min(clock.time[:]))/(max(clock.time[:]) - min(clock.time[:])) for time in clock.time[:]]

# plot the clock difference for each station --------------------------------------------------

for a in (range(len(clock.ant[:]))):
    xx = []
    yy = []
    difference = []

    for value in clock.val[:, a, 0]:
        xx.append(value * 1e9)

    for value in clock.val[:, a, 1]:
        yy.append(value * 1e9)
    
    if len(xx) == len(yy):
        for i in range(len(xx)):
            difference.append(xx[i] - yy[i])

    else:
        print('Cannot find difference as lists of XX and YY clock values have different lengths. Exiting...')
        sys.exit()
   
    plt.plot(normalised, difference, lw = 0.5, label = str(clock.ant[a])[2:-1])

# tidy up the plot ----------------------------------------------------------------------------

mpl.rcParams.update({'font.size': 10})
plt.xlim(min(normalised), max(normalised))
plt.title('Polarisation ' + str(clock.pol[0])[2:-1] + ' and ' + str(clock.pol[1])[2:-1] + ' clock difference versus time for all stations', fontsize = 10)
plt.xlabel('Time (normalised)', fontsize = 10)
plt.ylabel(str(clock.pol[0])[2:-1] + ' and ' + str(clock.pol[1])[2:-1] + ' clock difference (ns)', fontsize = 10)
plt.legend(loc = 'upper center', bbox_to_anchor = (0.5, -0.05), ncol = 17, frameon = False, fontsize = 7)
plt.savefig('clock_difference_xx_yy.png', bbox_inches = 'tight')
plt.close('all')

In [3]:
''' TEC versus time for XX ================================================================ '''

# import required modules ---------------------------------------------------------------------

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from losoto.h5parm import h5parm

# load the data -------------------------------------------------------------------------------

name = '/home/sean/test.h5'
f = h5parm(name, readonly = True)
tec = f.getSoltab('sol000', 'tec000')

# set up the plot -----------------------------------------------------------------------------

p = 0 # 0 = XX; 1 = YY
plt.figure(figsize = (24, 12))
normalised = [(time - min(tec.time[:]))/(max(tec.time[:]) - min(tec.time[:])) for time in tec.time[:]]

# plot the TEC values for each station --------------------------------------------------------

for a in (range(len(tec.ant[:]))):
    values = []

    for value in tec.val[:, a, p]:
        values.append(value)

    plt.plot(normalised, values, lw = 0.5, label = str(tec.ant[a])[2:-1])

# tidy up the plot ----------------------------------------------------------------------------

mpl.rcParams.update({'font.size': 10})
plt.xlim(min(normalised), max(normalised))
plt.title('TEC versus time for the ' + str(tec.pol[p])[2:-1] + ' polarisation for all stations', fontsize = 10)
plt.xlabel('Time (normalised)', fontsize = 10)
plt.ylabel(r'TEC ($10^{16} \cdot \mathrm{m}^{-2}$)', fontsize = 10)
plt.legend(loc = 'upper center', bbox_to_anchor = (0.5, -0.05), ncol = 17, frameon = False, fontsize = 7)
plt.savefig('tec_xx.png', bbox_inches = 'tight')
plt.close('all')

In [4]:
''' TEC difference between XX and YY versus time ========================================== '''

# import required modules ---------------------------------------------------------------------

import sys
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from losoto.h5parm import h5parm

# load the data -------------------------------------------------------------------------------

name = '/home/sean/test.h5'
f = h5parm(name, readonly = True)
tec = f.getSoltab('sol000', 'tec000')

# set up the plot -----------------------------------------------------------------------------

plt.figure(figsize = (24, 12))
normalised = [(time - min(tec.time[:]))/(max(tec.time[:]) - min(tec.time[:])) for time in tec.time[:]]

# plot the TEC difference for each station ----------------------------------------------------

for a in (range(len(tec.ant[:]))):
    xx = []
    yy = []
    difference = []

    for value in tec.val[:, a, 0]:
        xx.append(value)

    for value in tec.val[:, a, 1]:
        yy.append(value)
    
    if len(xx) == len(yy):
        for i in range(len(xx)):
            difference.append(xx[i] - yy[i])

    else:
        print('Cannot find difference as lists of XX and YY TEC values have different lengths. Exiting...')
        sys.exit()
   
    plt.plot(normalised, difference, lw = 0.5, label = str(tec.ant[a])[2:-1])

# tidy up the plot ----------------------------------------------------------------------------

mpl.rcParams.update({'font.size': 10})
plt.xlim(min(normalised), max(normalised))
plt.title('Polarisation ' + str(tec.pol[0])[2:-1] + ' and ' + str(tec.pol[1])[2:-1] + ' TEC difference versus time for all stations', fontsize = 10)
plt.xlabel('Time (normalised)', fontsize = 10)
plt.ylabel(str(tec.pol[0])[2:-1] + ' and ' + str(tec.pol[1])[2:-1] + r' TEC difference ($10^{16} \cdot \mathrm{m}^{-2}$)', fontsize = 10)
plt.legend(loc = 'upper center', bbox_to_anchor = (0.5, -0.05), ncol = 17, frameon = False, fontsize = 7)
plt.savefig('tec_difference_xx_yy.png', bbox_inches = 'tight')
plt.close('all')